### This is Example 4.3. Gambler’s Problem from Sutton's book.

A gambler has the opportunity to make bets on the outcomes of a sequence of coin flips. 
If the coin comes up heads, he wins as many dollars as he has staked on that flip; 
if it is tails, he loses his stake. The game ends when the gambler wins by reaching his goal of $100, 
or loses by running out of money. 

On each flip, the gambler must decide what portion of his capital to stake, in integer numbers of dollars. 
This problem can be formulated as an undiscounted, episodic, finite MDP. 

The state is the gambler’s capital, s ∈ {1, 2, . . . , 99}.
The actions are stakes, a ∈ {0, 1, . . . , min(s, 100 − s)}. 
The reward is zero on all transitions except those on which the gambler reaches his goal, when it is +1.

The state-value function then gives the probability of winning from each state. A policy is a mapping from levels of capital to stakes. The optimal policy maximizes the probability of reaching the goal. Let p_h denote the probability of the coin coming up heads. If p_h is known, then the entire problem is known and it can be solved, for instance, by value iteration.

In [1]:
import os, sys, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

if "../" not in sys.path:
  sys.path.append("../")


### Exercise 4.9 (programming)

Implement value iteration for the gambler’s problem and solve it for p_h = 0.25 and p_h = 0.55.



In [37]:
def value_iteration_for_gamblers(p_h, theta=0.0001, discount_factor=1.0):
    """
    Args:
        p_h: Probability of the coin coming up heads
    """
    rewards = np.zeros(101)
    rewards[100] = 1
    V = np.zeros(101)
    
    def one_step_lookahead(s, V, rewards):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            s: The gambler’s capital. Integer.
            V: The vector that contains values at each state. 
            rewards: The reward vector.
        Returns:
            A vector containing the expected value of each action. 
            Its length equals to the number of actions.
        """
        A = np.zeros(101)

        # NOTE: Minimum bet must be 1 - cannot bet 0 (no point doing so anyways).
        # This is how much we are betting.
        action_space = np.arange(1, min(s, 100 - s) + 1)
        for action in action_space:
            # Expected value of reward.
            # p_h (win) -> expected reward value of rewards[state + action].
            # 1 - p_h (lose) -> expected reward value of rewards[state - action].

            # Don't really need discount factor since this problem is undiscounted.
            A[action] = p_h * (rewards[s + action] + V[s + action]*discount_factor) + (1 - p_h) * (rewards[s - action] + V[s - action]*discount_factor)
        return A
    
    while True:
        delta = 0
        # Cycle through all states/capital and find best action.
        for state in range(1, 100):
            actions = one_step_lookahead(state, V, rewards)
            best_action_reward = max(actions)

            # Store largest change in reward for this iteration to terminate when convergence.
            delta = max(delta, best_action_reward - V[state])
            V[state] = best_action_reward
        
        if (delta < theta): break
    
    # Build policy from values.
    policy = np.zeros(100)
    for capital in range(1, 100):
        action = one_step_lookahead(capital, V, rewards)
        best_action = np.argmax(action)
        policy[capital] = best_action

    return policy, V

In [43]:
policy, v = value_iteration_for_gamblers(0.4)

print("Optimized Policy:")
print(policy)
print("")

print("Optimized Value Function:")
print(v)
print("")

Optimized Policy:
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 11. 15.  9. 17.
  7. 19. 20. 21. 22. 23. 24. 25.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.
 11. 12. 38. 39. 40.  9.  8. 43. 44. 45.  4. 47.  2.  1. 50.  1.  2.  3.
  4.  5.  6.  7.  8. 41. 10. 11. 12. 13. 14. 15. 34.  8. 18. 19. 20.  4.
 22.  2.  1. 25.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 12. 11.
 10.  9.  8.  7.  6.  5.  4.  3.  2.  1.]

Optimized Value Function:
[0.         0.00203162 0.00515507 0.00922512 0.01290418 0.01738208
 0.02306279 0.02781403 0.03227457 0.03767825 0.04346082 0.05035153
 0.05765757 0.06521897 0.06953507 0.07442925 0.08068842 0.08660695
 0.09421092 0.10313138 0.10865755 0.11596417 0.12587883 0.1335785
 0.1441471  0.16       0.16309304 0.16774251 0.17383767 0.17936474
 0.18607649 0.19459454 0.20172104 0.20841305 0.21652655 0.22519453
 0.2355273  0.24648826 0.25785582 0.2643026  0.27164589 0.28103263
 0.28991593 0.30131638 0.31471349 0.32298754 0.33394956 0.3488281
 0.36036974 0.

In [45]:
fig = px.bar(x=range(100), y=policy, 
             labels={ 'x': 'Capital', 'y': 'Stake'})
fig.update_layout(
    bargap=0.0,
    paper_bgcolor = 'black',
    plot_bgcolor = 'black',
    font=dict(color='white'),
    title_font_color='white'
)

fig.update_traces(marker_line_color='black')
fig.show()

In [51]:
# If only my stocks looked like this.
fig = px.line(x=range(100), y=v[:100], 
             labels={ 'x': 'Capital', 'y': 'Value'},
             color_discrete_sequence=['red'])
fig.update_layout(
    bargap=0.0,
    paper_bgcolor = 'black',
    plot_bgcolor = 'black',
    font=dict(color='white'),
    title_font_color='white'
)

fig.update_traces(marker_line_color='black')
fig.show()